In [ ]:
!git clone https://github.com/AssemblyAI-Examples/mediapipe-python.git
!pip install mediapipe
!pip install PyQt5
!pip install ipython==7.32.0
!pip install onnxruntime

Cloning into 'mediapipe-python'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 45 (delta 14), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (45/45), 31.45 MiB | 23.10 MiB/s, done.
Resolving deltas: 100% (14/14), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.9/793.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00


In [ ]:
import cv2 as cv
from mediapipe import solutions as mp
import time
import numpy as np
import onnxruntime as ort
from tensorflow.keras.models import load_model
from PIL import Image
import math


Prepare Image

In [ ]:
def preprocess_image(image):
    hands = mp.hands.Hands()

    # Convert the image to RGB format
    image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    try:
        results = hands.process(image_rgb)
        if results.multi_hand_landmarks:
            x_values = []  # List to store X values of landmarks
            y_values = []
            for hand_landmarks in results.multi_hand_landmarks:
                for landmark_id, landmark in enumerate(hand_landmarks.landmark):
                    # Extracting X, Y, Z coordinates of each landmark
                    x = int(landmark.x * image.shape[1])
                    x_values.append(x)
                    y = int(landmark.y * image.shape[0])
                    y_values.append(y)
                    z = landmark.z  # Z coordinate may not be available in all cases

    except Exception as err:
        print(f"Error processing the image: {err}")

    mean_x = sum(x_values) / len(x_values)
    mean_y = sum(y_values) / len(y_values)

    furthest_distance = 0
    for x, y in zip(x_values, y_values):
        distance = math.sqrt((x - mean_x)**2 + (y - mean_y)**2)
        furthest_distance = max(furthest_distance, distance)

    # Calculate bounding box dimensions
    left = int(mean_x - furthest_distance - 10)
    top = int(mean_y - furthest_distance - 10)
    right = int(mean_x + furthest_distance + 10)
    bottom = int(mean_y + furthest_distance + 10)

    # Crop the image
    cropped_image = image[top:bottom, left:right]

    # Resize the image
    resized_image = cv.resize(image, (50, 50))

    # Convert to Array
    img_array = np.expand_dims(resized_image, axis=0)

    # Normalize pixel values from 0 to 1
    img_array = img_array / 255.0

    return img_array


In [ ]:
# Path to image to test
image_path = "B.jpg"
test_img = cv.imread(image_path)
input_image = preprocess_image(test_img)

Test Image



In [ ]:
model = load_model('final_model.h5')
predictions = model.predict(input_image)
class_index = np.argmax(predictions)

print(f"Predicted Class Index: {class_index}")
map_characters1 = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'del', 27: 'nothing', 28: 'space'}
print(map_characters1[class_index])
combined_dict = dict(zip(map_characters1.values(), predictions[0]))

# Sort
sorted_items = sorted(combined_dict.items(), key=lambda x: x[1], reverse=True)
print(sorted_items)
# Print the top 3 items
top_3_items = sorted_items[:3]
for key, value in top_3_items:
  print(f"{key}: {value}")

correct_letter = "B"
letters = [key for key, _ in top_3_items]
correct = correct_letter in letters

print(correct)


1/1 [==============================] - 0s 317ms/step
Predicted Class Index: 13
N
[('N', 0.9251515), ('M', 0.04631616), ('E', 0.026110524), ('I', 0.00078997994), ('D', 0.00063590857), ('T', 0.0003558675), ('V', 0.0003491655), ('P', 8.6636355e-05), ('S', 6.969405e-05), ('space', 4.922691e-05), ('G', 3.1507145e-05), ('O', 2.8487799e-05), ('R', 1.0410684e-05), ('B', 8.319084e-06), ('L', 2.0140537e-06), ('nothing', 1.4477303e-06), ('K', 9.3799326e-07), ('X', 8.0206786e-07), ('W', 6.104232e-07), ('A', 3.2597256e-07), ('Z', 2.6067931e-07), ('U', 6.209043e-08), ('F', 1.7504929e-08), ('J', 5.223402e-09), ('Q', 3.711153e-09), ('C', 2.806035e-10), ('del', 2.6406302e-10), ('Y', 2.4872932e-11), ('H', 1.3908952e-15)]
N: 0.925151526927948
M: 0.04631616175174713
E: 0.02611052431166172
False


Find Nodes

In [ ]:
def find_nodes(image):
  mp_hands = mp.solutions.hands
  my_hand_drawing = mp.solutions.drawing_utils

  hands = mp_hands.Hands()

  # Flip the image
  # Have to switch colors because openCV color scheme is BGR not RGB
  image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)

  # Store the results
  start_time = time.time()
  try:
      results = hands.process(image_rgb)
  except Exception as err:
      print(f"Error processing the image: {err}")
  height, width, _ = image.shape

  # Create an empty image
  new_image = np.zeros((height, width, 3), dtype=np.uint8)  # Replace height and width with the desired image dimensions

  # Draw landmarks on the image
  for landmarks in results.multi_hand_landmarks:
          #Adds the landmarks (lines) on the hands
          my_hand_drawing.draw_landmarks(
          new_image,
          landmarks, mp_hands.HAND_CONNECTIONS
      )

  #new_image = cv.flip(new_image, 1)

  image_bgr = cv.cvtColor(cv.flip(image_rgb, 1), cv.COLOR_RGB2BGR)

  return new_image